In [15]:
from json_extractor import from_file_get_n_docs
import json
from similarity import similar, map_list_a_to_b, best_match_list
from collections import defaultdict

# Get Parsed docs

In [2]:
STANFORD_FILENAME = '100_parsed_from_stanford.jsonl'
TAGME_FILENAME    = '100_parsed_from_tagme.jsonl'
WIKIFIER_FILENAME = '100_parsed_from_wikifier.jsonl'

In [3]:
stanford_parsed_docs = from_file_get_n_docs(STANFORD_FILENAME, 5)

In [4]:
wikifier_parsed_docs = from_file_get_n_docs(WIKIFIER_FILENAME, 5)

In [5]:
tagme_parsed_docs    = from_file_get_n_docs(TAGME_FILENAME, 5)

In [11]:
test_index = 0
stanford_doc = stanford_parsed_docs[test_index]
wikifier_doc = wikifier_parsed_docs[test_index]
tagme_doc = tagme_parsed_docs[test_index]

# Generate Word Dicts

## Common token list + tagged-words dictionary => tagged-words list

#### Getting a word annotation from a token_index, a map and a tagged_words map

In [21]:
def merge_dicts(x, y):
    '''Given two dicts, merge them into a new dict as a shallow copy.'''
    z = x.copy()
    z.update(y)
    
    return z

add_color_to_map = lambda c, m: {int(k): merge_dicts({'color': c}, v) for k, v in m.items()}

no_match_color = {'color': 'white'}
word_tag_from_index_and_maps = lambda i, atob, b: b.get(atob[i], no_match_color)

def tag_tokens_with_linked_doc(tokens, linker_doc, color="royalblue"):
    tokens_to_linker = map_list_a_to_b(tokens, linker_doc['tokens'])
    
    tagged_words = add_color_to_map(color, linker_doc['tagged-words'])
    tags = [word_tag_from_index_and_maps(i, tokens_to_linker, tagged_words) for (i, word) in enumerate(tokens)]
    
    return tags

#### Stanford to Wikifier

In [23]:
stanford_to_wikifier = tag_tokens_with_linked_doc(stanford_doc['tokens'], wikifier_doc)

In [45]:
stanford_to_wikifier[2]

{'color': 'royalblue',
 u'cosine': 0.2378448458365748,
 u'dbPediaIri': u'http://dbpedia.org/resource/Worcester',
 u'dbPediaTypes': [u'City', u'Settlement', u'PopulatedPlace', u'Place'],
 u'secLang': u'en',
 u'secTitle': u'Worcester',
 u'secUrl': u'http://en.wikipedia.org/wiki/Worcester'}

#### Stanford to Tagme

In [33]:
stanford_to_tagme = tag_tokens_with_linked_doc(stanford_doc['tokens'], tagme_doc, 'tomato')

In [46]:
stanford_to_tagme[2], stanford_doc['tokens'][2]

({'color': 'tomato',
  u'dbpedia_categories': [u'Worcester',
   u'Populated places on the River Severn',
   u'County towns in England',
   u'Local government in Worcestershire',
   u'Non-metropolitan districts of Worcestershire',
   u'Local government districts of the West Midlands (region)',
   u'Post towns in the WR postcode area',
   u'Cities in the West Midlands (region)'],
  u'end': 2,
  u'id': 58681,
  u'link_probability': u'0.26996',
  u'rho': u'0.18800',
  u'spot': u'Worcester',
  u'start': 2,
  u'title': u'Worcester'},
 u"Worcester's")